In [1]:
import btk
import numpy as np
import pickle

from collections import Counter
from tqdm import tqdm

In [2]:
version = 'x5'
with open(f'D:/dstore/nlp/w2v/wlst-{version}', 'rb') as f:
    wlst = pickle.load(f)

with open(r'D:\dstore\nlp\w2v\wcounts', 'rt', encoding='utf8') as f:
    bwlst = Counter({f'_{x.strip().split()[1]}_': int(x.strip().split()[0]) for x in f.readlines() if int(x.strip().split()[0]) > 9})

with open(f'D:/dstore/nlp/w2v/uwlst', 'rb') as f:
    cnts = pickle.load(f)

for n in range(3, 8):
    scale = (np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]) ** (3.333 / (2 + (n/4)))) / 2
    for x in cnts:
        if len(x) == n:
            cnts[x] = int(cnts[x] / scale)
for x in cnts:
    if cnts[x] < 1:
        cnts[x] += 1

with open(f'D:/dstore/nlp/w2v/bwlst', 'rb') as f:
    bcnts = pickle.load(f)

affs = set()

with open(r'D:\dstore\nlp\w2v\prefs2', 'rt', encoding='utf8') as f:
    affs = {x.strip() for x in f.readlines()}

with open(r'D:\dstore\nlp\w2v\suffs2', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        affs.add(x.strip())

esuf = ('d_', 'r_', 's_', 'n_', "'d_", "'s_", "'n_", "'t_", 'er_', 'ed_', 'es_')
doubles = ('ll', 'tt', 'pp', 'mm', 'nn', 'bb', 'ss', 'dd', 'gg')
ysufs = ('ifies_', 'ities_', 'ies_', 'ied', 'iful_', 'iness_')

def fragment(word: str):
    if word in wlst:
        return [word]
    queue = []
    finished = []
    keys = []
    for suf in esuf:
        if suf in word:
            k = ([suf], word.replace(suf, '_'))
            keys.append(k)
            queue.append((*k, affs.copy()))
            finished.append(tuple(sorted([suf, word.replace(suf, '_')])))
    for item in queue[::-1]:
        for suf in esuf:
            if suf in item[1]:
                k = (sorted([*item[0], suf]), item[1].replace(suf, '_'))
                if k not in keys:
                    keys.append(k)
                    queue.append((*k, item[2]))
                fin = tuple(sorted([item[0][0], suf, item[1].replace(suf, '_')]))
                if fin not in finished:
                    finished.append(fin)
    queue.append(([], word, affs.copy()))
    while queue:
        item = queue.pop(0)
        found = False
        for affix in item[2]:
            if affix in item[1]:
                found = True
                if affix in ysufs:
                    k = (sorted([*item[0], affix]), item[1].replace(affix, 'y_'))
                else:
                    k = (sorted([*item[0], affix]), item[1].replace(affix, '_'))
                if k not in keys:
                    if k[1][1:3] in doubles:
                        mod = list(k[1])
                        mod.pop(1)
                        k = (k[0], ''.join(mod))
                    if k[1][-3:-1] in doubles:
                        mod = list(k[1])
                        mod.pop(-2)
                        k = (k[0], ''.join(mod))
                    aff_copy = item[2].copy()
                    keys.append(k)
                    aff_copy.remove(affix)
                    queue.append((*k, aff_copy))
                fin = [x for x in k[0]]
                fin.append(k[1])
                fin = tuple(sorted(fin))
                if fin not in finished:
                    finished.append(fin)
        if not found:
            fin = [x for x in item[0]]
            fin.append(item[1])
            fin = tuple(sorted(fin))
            if fin not in finished:
                finished.append(fin)
    flen = len(finished) - 1
    for i, x in enumerate(finished[::-1]):
        xscore = 0
        for y in x:
            if y.strip('_') in cnts:
                xscore += cnts[y.strip('_')]
        finished[flen-i] = (xscore, x)
    finished = sorted(finished, key=lambda x: x[0], reverse=True)
    outp = []
    for x in finished[:4]:
        outp.extend(x[1])
    return outp


In [ ]:
short_words = [x for x in bwlst.most_common() if len(x[0]) < 6 and x[1] > 256]

In [ ]:
[x for x in bcnts.most_common() if len(x[0]) == 4 and x[0][0] == '_']

In [ ]:
[x for x in bwlst.most_common() if len(x[0]) == 5 and x[0][0] == '_' and x[0][-1] == '_']

In [20]:
caffs = {x.strip('_') for x in affs}
len([x for x in final_frags.most_common() if x[0].strip('_') not in caffs and len(x[0].strip('_')) > 2])

155555

In [ ]:
tword = '_bacteriophage_'
fragment(tword)

In [ ]:
c = np.random.randint(0, len(bwlst) - 101)
[fragment(x[0]) for x in bwlst.most_common()[c:c+20]]

In [ ]:
fix = 'dom'
targ = [x for x in bwlst.most_common() if fix in x[0]]

tlen = len(fix)
ncount = Counter()

for x in targ:
    if fix[0] == '_':
        ncount[x[0][:tlen+1]] += x[1] * len([y for y in targ if x[0][:tlen+1] in y[0]])
    else:
        ncount[x[0][len(x[0]) - 1 - tlen:]] += x[1] * len([y for y in targ if x[0][len(y[0]) - 1 - tlen:] in y[0]])

for x in ncount.most_common()[:8]: print(x)
print('\n')

for x in targ[:64]:
    print(x)

In [11]:
with open(f'D:/dstore/nlp/w2v/uwlst', 'rb') as f:
    cnts = pickle.load(f)

for n in range(3, 8):
    print(np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]))
print('\n')
for n in range(3, 8):
    print((np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]) ** (3.333 / (2 + (n/4))) / 2))

323.2823342054374
42.49057119965607
11.874224257866846
5.461737040443904
2.6007610127323284


550.2795276912724
32.21091404182568
6.324395425954965
2.518370244579201
1.1692605061876833


In [ ]:
cnts.most_common()[:64]

In [ ]:
[x for x in bcnts.most_common() if len(x[0]) > 2 and x[0] not in affs][:64]

In [ ]:
[(x[0], x[0].replace('ically_', '')) for x in wlst.most_common() if 'ically' in x[0]]